In [ ]:
#Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path =  '/content/drive/MyDrive/Soil types'

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
# Image dimensions
img_width, img_height = 150, 150

In [ ]:
# Parameters for training
batch_size = 32
epochs = 20

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # Splitting the data into training and validation

In [ ]:
train_generator = train_datagen.flow_from_directory(
    file_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')  # set as training data

Found 251 images belonging to 5 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    file_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  # set as validation data

Found 61 images belonging to 5 classes.


In [ ]:
# Model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(5, activation='softmax')  # 5 classes: Black, Cinder, Laterite, Peat, Yellow
])

In [ ]:
# Model compilation
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/20
7/7 [==============================] - 8s 1s/step - loss: 1.8470 - accuracy: 0.1872 - val_loss: 1.5857 - val_accuracy: 0.1562
Epoch 2/20
7/7 [==============================] - 8s 1s/step - loss: 1.5282 - accuracy: 0.3151 - val_loss: 1.3828 - val_accuracy: 0.3750
Epoch 3/20
7/7 [==============================] - 8s 1s/step - loss: 1.2145 - accuracy: 0.4703 - val_loss: 1.0504 - val_accuracy: 0.5000
Epoch 4/20
7/7 [==============================] - 7s 963ms/step - loss: 0.8165 - accuracy: 0.7032 - val_loss: 0.4895 - val_accuracy: 0.9062
Epoch 5/20
7/7 [==============================] - 7s 982ms/step - loss: 0.6605 - accuracy: 0.7580 - val_loss: 0.6420 - val_accuracy: 0.7812
Epoch 6/20
7/7 [==============================] - 8s 1s/step - loss: 0.6575 - accuracy: 0.7489 - val_loss: 0.7660 - val_accuracy: 0.6562
Epoch 7/20
7/7 [==============================] - 8s 1s/step - loss: 0.4986 - accuracy: 0.8037 - val_loss: 0.7117 - val_accuracy: 0.7812
Epoch 8/20
7/7 [===================

In [ ]:
# Model evaluation
loss, accuracy = model.evaluate(validation_generator, verbose=1)
print(f'Validation loss: {loss:.4f}')
print(f'Validation accuracy: {accuracy:.4f}')


2/2 [==============================] - 1s 294ms/step - loss: 0.3125 - accuracy: 0.9016
Validation loss: 0.3125
Validation accuracy: 0.9016


In [ ]:
# Save the model
model.save('soil_classification_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model('soil_classification_model.h5')

# Function to preprocess uploaded image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(img_width, img_height))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0  # Normalize the image
    return img

# Function to make predictions
def predict_soil_type(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    if predicted_class == 0:
        return 'Black Soil'
    elif predicted_class == 1:
        return 'Cinder Soil'
    elif predicted_class == 2:
        return 'Laterite Soil'
    elif predicted_class == 3:
        return 'Peat Soil'
    elif predicted_class == 4:
        return 'Yellow Soil'
    else:
        return 'Unknown'

# Upload an image in Colab
from google.colab import files

uploaded = files.upload()

# Make predictions for uploaded images
for filename in uploaded.keys():
    image_path = filename
    prediction = predict_soil_type(image_path)
    print(f'The predicted soil type for {filename} is: {prediction}')


Saving Cinder_Soil_ (8).jpg to Cinder_Soil_ (8).jpg
1/1 [==============================] - 0s 63ms/step
The predicted soil type for Cinder_Soil_ (8).jpg is: Cinder Soil


In [ ]:
!pip install django


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 27.9 MB/s eta 0:00:00


In [ ]:
!django-admin startproject myproject


In [ ]:
%cd myproject


/content/myproject


In [ ]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

app = Flask(__name__)
model = load_model('/content/soil_classification_model.h5')

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    return img

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})
    img_path = 'temp.jpg'
    file.save(img_path)
    img = preprocess_image(img_path)
    prediction = model.predict(img)
    predicted_class = np.argmax(prediction)
    soil_types = ['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil']
    predicted_soil_type = soil_types[predicted_class]
    return jsonify({'predicted_soil_type': predicted_soil_type})

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
%%writefile index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Soil Type Predictor</title>
</head>
<body>
    <h1>Soil Type Predictor</h1>
    <input type="file" id="fileInput">
    <button onclick="predict()">Predict</button>
    <div id="result"></div>

    <script>
        function predict() {
            var fileInput = document.getElementById('fileInput');
            var file = fileInput.files[0];
            var formData = new FormData();
            formData.append('file', file);

            fetch('/predict', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                document.getElementById('result').innerText = 'Predicted Soil Type: ' + data.predicted_soil_type;
            })
            .catch(error => {
                console.error('Error:', error);
            });
        }
    </script>
</body>
</html>


Writing index.html


In [ ]:
!python manage.py runserver


Watching for file changes with StatReloader
Performing system checks...

System check identified no issues (0 silenced).

You have 18 unapplied migration(s). Your project may not work properly until you apply the migrations for app(s): admin, auth, contenttypes, sessions.
Run 'python manage.py migrate' to apply them.
February 12, 2024 - 04:40:49
Django version 5.0.2, using settings 'myproject.settings'
Starting development server at http://127.0.0.1:8000/
Quit the server with CONTROL-C.

